# Stress Classifier
## Overview
1. [Select Database](#1-select-database)
2. [Import Libraries and Load Useful Utilities](#2-import-libraries-and-load-useful-utilities)
3. [Data Extraction](#3-data-extraction): Downloads and sorts through databases.
4. [Pre-processing](#4-pre-processing): filtering and signal cleaning.
5. [Feature Extraction](#5-feature-extraction): R-R peaks, PQRST peaks, EDR, in addition to mean, kurtosis etc.
6. [Feature Selection](#6-feature-selection): Visualise labelled feature distribution, select desired labels etc.
7. [Traditional Machine Learning Methods](#7-traditional-machine-learning-methods): Linear classification including Random Forests, LDA, Bagged Trees and linear/non-linear SVM.
8. [Convolutional Neural Network](#8-convolutional-neural-network): 1-D convolutional neural network with automatic feature extraction 

## 1. Select Database
Select between:
* [Spider-Fearful ECG](https://doi.org/10.1371/journal.pone.0231517) 
* BrainPatch ECG

In [82]:
# DATABASE = 'Spider'
DATABASE = 'BrainPatch'

## 2. Import Libraries and Load Useful Utilities
Please run `pip install -r requirements.txt` first

In [83]:
# Import necessary modules
import pandas as pd
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback


import multiprocessing as mp
import math
from datetime import datetime, timedelta
import os
import pickle
import sys
import subprocess
import warnings
from itertools import product
from typing import List
import IPython.display as display

In [84]:
# Useful Utilities
class Utilities():
    def __init__(self):    
        pass

    def progress_bar(current_message, current, total, bar_length=20):
        fraction = current / total
        arrow = int(fraction * bar_length - 1) * '-' + '>'
        padding = int(bar_length - len(arrow)) * ' '
        ending = '\n' if current == total else '\r'
        print(f'{current_message}: [{arrow}{padding}] {int(fraction*100)}%', end=ending)


    def check_csv_exists(folder_path, sample_index):
        # read the CSV file into a dataframe and append to the list
        filename = os.path.join(folder_path, f'df_{index}.csv')
        try:
            df = pd.read_csv(filename)
        except FileNotFoundError:
            return False
        return filename


    def load_dataframe(filename):
        # read the CSV file into a dataframe and append to the list
        df = pd.read_csv(filename)
        return df
    

    def save_dataframe_list(list_of_dfs:List[pd.DataFrame], folder_path:str, file_name:str):
        # create directoy if necessary
        os.makedirs(folder_path, exist_ok=True) 
        for i, df in enumerate(list_of_dfs):
            file_path = f"{folder_path}/{file_name}_{i}.csv"
            df.to_csv(file_path, index=False)


    def save_dataframe(df:pd.DataFrame, folder_path:str, file_name:str):
        print(f"Saving Dataframe to: {folder_path}/{file_name}.csv...", end='')
        # create directoy if necessary
        os.makedirs(folder_path, exist_ok=True) 
        df.to_csv(f'{folder_path}/{file_name}.csv', index=False)
        print("Saved.")

In [85]:
# For bigger plots
plt.rcParams['figure.figsize'] = [10, 6]  
plt.rcParams['font.size'] = 14

## 3. Data Extraction
* Downloads data, normalizes timeframe, attaches labels, and saves sorted data.

In [86]:
# This class handles downloading and sorting the Spider database.
# Method from this paper https://doi.org/10.1371/journal.pone.0231517
class SpiderDataExtraction():
    def __init__(self, directory):
        self.directory = directory + 'Spider'
        self.sampling_frequency = 100
        self.sorted_ECG = pd.DataFrame(columns=['Timestamp', 'ECG', 'Stress Level'])

    
    # Downloads and extracts database 
    def download_data(self):
        if sys.platform == 'linux':
            url = 'https://physionet.org/files/ecg-spider-clip/1.0.0/'
            print("Downloading database...this may take a while")
            os.makedirs(self.directory, exist_ok=True)
            cmd = f"wget -r -N -c -np -P {self.directory} {url}"
            subprocess.run(cmd)
        else:
            sys.exit("Unable to download database. If you are running Windows/Mac please download manually via https://physionet.org/content/ecg-spider-clip/1.0.0/")
    

    # sorts data from each participant, labelling each ECG recording and appends to one dataframe.
    # Following the SB approach in the study.
    def sort_data(self):       
        directory = self.directory + '/physionet.org/files/ecg-spider-clip/1.0.0/'
        
        # Exclude VP70 because of noise
        sub_directories = ['VP02', 'VP03','VP05','VP06','VP08','VP09','VP11','VP12','VP14','VP15','VP17','VP18','VP20','VP23','VP24','VP26','VP27',
                'VP29','VP30','VP32','VP33','VP35','VP36','VP38','VP39','VP41','VP42','VP44','VP45','VP47','VP48','VP50','VP51','VP53',
                'VP54','VP56','VP57','VP59','VP61','VP62','VP63','VP64','VP65','VP66','VP68','VP69','VP71','VP72','VP73','VP74',
                'VP75','VP76','VP77','VP78','VP79','VP80']
        
        # Path to Ratings file for all particpants
        subjective_ratings_file = f'{self.directory}/Subjective Ratings.txt'

        # Read in the subject ratings file (ignore arousal markers, interested in angst)
        ratings_df = (
            pd.read_csv(subjective_ratings_file, sep='\t', names=['Subject','Group','Session', '4', '8', '12', '16', 'NA1', 'NA2', 'NA3', 'NA4'], encoding='UTF-16')
            .drop(columns=['NA1', 'NA2', 'NA3', 'NA4'])
            .iloc[1:]
            .reset_index(drop=True)
            .astype(int)
        )

        for index, sub_directory in enumerate(sub_directories):
            Utilities.progress_bar('Sorting database', index, len(sub_directories)-1)

            # set participant data paths
            ECG_file = f'{directory}{sub_directory}/BitalinoECG.txt'
            triggers_file = f'{directory}{sub_directory}/Triggers.txt'

            # Get participant number
            participant_no = int(sub_directory[2:])

            # read in particpant ECG raw data file and reorder to get columns Timestamp, ECG 
            raw_df = pd.read_csv(ECG_file, sep='\t', names = ['ECG','Timestamp','NA'])
            raw_df = raw_df.drop(columns=['NA'])
            raw_df = raw_df[['Timestamp', 'ECG']]
            
            # Read in participant trigger file
            triggers_df = pd.read_csv(triggers_file, sep='\t', names = ['Clip','On','Off'])

            # Determine stress levels by correspoding the raw ecg data with the triggers file and the Subjective Ratings file.
            # Iterate through the 16 stress clips (first clip is a demo):
            for i in range(1, 17):
                # Determine row in ratings file
                row = ratings_df.loc[ratings_df['Subject'] == participant_no].index[0]

                # find stress for the clip in the ratings file
                stress_level = ratings_df.iloc[row]['4'] if i <= 4 else ratings_df.iloc[row]['8'] if i <= 8 else ratings_df.iloc[row]['12'] if i <= 12 else ratings_df.iloc[row]['16']

                # convert stress level to Low, Medium or High (1-3)
                stress_level = 2 if (stress_level <= 2) else 3

                # Get 60 second slice of ECG data for that clip
                clip_start_time = triggers_df.iloc[i]['On']
                start_index = raw_df.index[raw_df['Timestamp']>clip_start_time].tolist()[0]
                clip_df = raw_df.iloc[start_index:start_index + (self.sampling_frequency * 60)].copy(deep=False)
                clip_df['Stress Level'] = stress_level
                self.sorted_ECG = pd.concat([self.sorted_ECG, clip_df], axis=0, ignore_index=True)

            # Add the last 3 minute resting phase (stress level Low) to the data
            rest_start_time = triggers_df.iloc[-1]['On']
            start_index = (raw_df['Timestamp'] > rest_start_time).idxmin() + (self.sampling_frequency * 120)
            rest_df = raw_df.iloc[start_index: start_index + (self.sampling_frequency * 180)].copy(deep=False)
            rest_df.loc[:, 'Stress Level'] = 1
            self.sorted_ECG = pd.concat([self.sorted_ECG, rest_df], axis=0, ignore_index=True)



In [87]:
# This class handles sorting through the data collected using the BrainPatch ECG Recorder.
class BPDataExtraction():
    def __init__(self, directory):
        self.directory = directory + 'BrainPatch/'
        self.sorted_ECG = pd.DataFrame(columns=['Timestamp', 'ECG', 'Stress Level'])

    
    # interpolates data to achieve sampling rate
    def interpolate(self, sample:pd.DataFrame) -> pd.DataFrame:
        # convert timestamp column to a NumPy array
        timestamps = sample['Timestamp'].to_numpy()

        # calculate the time difference between each pair of adjacent timestamps
        time_diff = np.diff(timestamps)

        # calculate the average sampling rate of the data
        sampling_frequency = (1 / np.mean(time_diff))*1e6

        interpolated_sample = pd.DataFrame(columns=['Timestamp', 'ECG'])

        # interpolate the data to obtain desired sampling rate
        interpolated_sample['ECG'] = nk.signal_resample(sample['ECG'], sampling_rate=sampling_frequency, desired_sampling_rate=self.sampling_frequency)
        time_interval_us = (1/self.sampling_frequency)*1e6
        interpolated_sample['Timestamp'] = [i*time_interval_us for i in range(len(interpolated_sample['ECG']))]

        return interpolated_sample.astype('int64')


    # define a function to map stress levels to labels
    def map_stress_level(self, level):
        if level >= 7:
            return 3
        elif level >= 5:
            return 2
        else:
            return 1


     # Sorts data from each participant, interpolates it (downsampling), labelling each ECG recording and appends to one dataframe.
    def sort_data(self, sampling_frequency:int=1000):
        self.sampling_frequency = sampling_frequency
        # list the contents of the directory and filter out subfolders
        recordings = [f for f in os.listdir(self.directory) if os.path.isfile(os.path.join(self.directory, f))]

        for index, recording in enumerate(recordings):
            Utilities.progress_bar('Sorting database', index, len(recordings)-1)
            # get recording number
            split_string = recording.split(' ')
            second_element = split_string[1]
            second_split = second_element.split('.')
            recording_number = int(second_split[0])

            # extract recording dataframe
            recording_df = pd.read_csv(self.directory + recording, skiprows=2, usecols=[0, 1], header=None, names=['Timestamp', 'ECG'])
            # interpolate it to desired sampling frequency
            recording_df = self.interpolate(recording_df)
            # load events containing the stress level markers
            events_df = pd.read_csv(self.directory + 'Events/Events.csv')
            # extract events for that recording
            events_df = events_df[events_df['Recording No'] == recording_number].dropna(how='all', axis=1)
            # convert string values to datetime objects
            events_df['Timestamp'] = pd.to_datetime(events_df['Timestamp'], format='%H:%M')
            # normalise timestamps 
            events_df['Timestamp'] = events_df['Timestamp'] - events_df['Timestamp'].iloc[0]
            # convert from nanoseconds to microseconds
            events_df['Timestamp'] = (events_df['Timestamp']/ 1e3).astype('int64')
            # label recording with Events label
            recording_df = pd.merge_asof(recording_df, events_df, on='Timestamp', direction='backward')
            recording_df = recording_df.drop('Recording No', axis=1)
            # convert stress level to Low(1-4), Medium(5-6) or High(7-10)
            recording_df['Stress Level'] = recording_df['Stress Level'].apply(self.map_stress_level)
            # concatentate to main dataframe
            self.sorted_ECG = pd.concat([self.sorted_ECG, recording_df], axis=0, ignore_index=True)

In [88]:
data_path = '/data/myenv/Data/'

if DATABASE == 'Spider':
    sde = SpiderDataExtraction(data_path)
    # sde.download_data()
    sde.sort_data()
    dataframe_path = f'{data_path}Spider/Dataframes'
    Utilities.save_dataframe(sde.sorted_ECG, dataframe_path, 'Sorted')

elif DATABASE == 'BrainPatch':
    bpde = BPDataExtraction(data_path)
    bpde.sort_data(sampling_frequency=1000) # downsamples to 1kHz
    dataframe_path = f'{data_path}BrainPatch/Dataframes'
    Utilities.save_dataframe(bpde.sorted_ECG, dataframe_path, 'Sorted')

KeyboardInterrupt: 

## 4. Pre-Processing
* Interpolates data to sample rate using timestamps 
* Semgents data using rolling window with overlap
* Cleans data using Neurokit's 5th Order Butterworth filter.

In [ ]:
class PreProcessing():
    def __init__(self, sorted_ECG:pd.DataFrame, sampling_frequency:int):
        self.sorted_ECG = sorted_ECG
        self.sampling_frequency = sampling_frequency
       

    # segments data with overlap using rolling window, if semgnet_heartbeats true, then segment is centralised around the heartbeat (R peak)
    def segment(self, window_length_s:float, overlap:float, segment_hearbeats=False):
        # convert window_length in seconds to samples
        self.window_samples = int(window_length_s * self.sampling_frequency)
        # Calculate the step_size as the fraction of the total window samples
        step_size = int(self.window_samples * (1-overlap)) 

        # Initialize starting variables
        current_index = 0
        current_stressed = self.sorted_ECG['Stress Level'][current_index]
        self.preprocessed_ECG = pd.DataFrame(columns=['Timestamp', 'ECG', 'Stress Level'])

        # faster to concatenate at the end 
        preprocessed_ECG_list = []

        # get all R peaks index if required
        if segment_hearbeats:
            r_peaks = nk.ecg_peaks(self.sorted_ECG['ECG'], sampling_rate=self.sampling_frequency)
        
        # Loop through the entire dataframe
        while current_index < len(self.sorted_ECG['ECG']):  
            Utilities.progress_bar('Segmenting data', current_index, len(self.sorted_ECG))
            # calculate end index in window and exit if out of bounds          
            end_index = current_index + self.window_samples
            if (end_index > len(self.sorted_ECG['ECG'])):
                break
            
            # Check if the window overlaps a different label
            end_stressed = self.sorted_ECG['Stress Level'][end_index]

            # If the next window has a different label, skip to next start of next label
            if end_stressed != current_stressed:
                while (current_stressed == self.sorted_ECG['Stress Level'][current_index]):
                    current_index += 1
                current_stressed = end_stressed

            # otherwise, add segment to list of pre-processed ECG
            else:
                if segment_hearbeats:
                    # get index of next r peak
                    while not bool(r_peaks[0]['ECG_R_Peaks'][current_index]):
                        current_index += 1
                    # append segment centred on r-peak to dataframe
                    preprocessed_ECG_list.append(self.sorted_ECG.iloc[(current_index - (self.window_samples//2)):(current_index + (self.window_samples//2))].astype('Float64'))
                    # shift the window to next non r-peak index
                    current_index += 1
                else:
                    # append segment to dataframe
                    preprocessed_ECG_list.append(self.sorted_ECG.iloc[current_index:current_index + self.window_samples].astype('Float64'))
                    # Shift the window
                    current_index += step_size
            
        self.preprocessed_ECG = pd.concat(preprocessed_ECG_list, axis=0, ignore_index=True).astype('Float64')
        Utilities.progress_bar('Segmenting data', current_index, current_index)
    
    
    def create_2d(self):
        # convert the pandas DataFrame into a 2D pandas where each row has the size of window and the corresponding label (stress level)
        # self.preprocessed_ECG_2d = self.preprocessed_ECG.groupby(self.preprocessed_ECG.index // self.window_samples).apply(lambda x: pd.Series({'ECG': x['ECG'].values, 'Stress Level': x['Stress Level'].values[0]}))
        
        # Calculate the number of rows required
        num_rows = len(self.preprocessed_ECG['ECG']) // self.window_samples

        # Create an empty dataframe to hold the reshaped data
        df_reshaped = pd.DataFrame(index=range(num_rows), columns=[f"ECG {i}" for i in range(self.window_samples)])

        # Reshape the data
        for i in range(num_rows):
            start_idx = i * self.window_samples
            end_idx = (i + 1) * self.window_samples
            values = self.preprocessed_ECG['ECG'].iloc[start_idx:end_idx].values
            df_reshaped.iloc[i, :] = values
        
        self.preprocessed_ECG_2d = df_reshaped
        self.preprocessed_ECG_2d['Stress Level'] = self.preprocessed_ECG['Stress Level'][::self.window_samples].reset_index(drop=True)


    def clean(self):
        # Clean each sample in the stressed and not stressed data (overwrites original data)
        # using method 'neurokit' (0.5 Hz high-pass butterworth filter (order = 5), followed by powerline filtering) but can be changed to other cleaning methods
        print("Cleaning data...")
        self.preprocessed_ECG['ECG'] = pd.Series(nk.ecg_clean(self.preprocessed_ECG['ECG'], self.sampling_frequency, method='neurokit')).astype('Float64')

In [ ]:
window_length_s = 20 # window length for each segment
overlap = 0.1 # overlap for sliding window

if DATABASE == 'Spider':
    pp = PreProcessing(sde.sorted_ECG, sde.sampling_frequency)
    pp.segment(window_length_s, overlap)
    pp.clean()
    Utilities.save_dataframe(pp.preprocessed_ECG, dataframe_path, 'Preprocessed')

elif DATABASE == 'BrainPatch':
    pp = PreProcessing(bpde.sorted_ECG, 1000) # the data will be downsampled to 1000Hz
    pp.segment(window_length_s, overlap)
    pp.clean()
    Utilities.save_dataframe(pp.preprocessed_ECG, dataframe_path, 'Preprocessed')


Segmenting data: [------------------->] 100%
Cleaning data...
Saving Dataframe to: /data/myenv/Data/BrainPatch/Dataframes/Preprocessed.csv...Saved.


## 5. Feature Extraction
* Extracts features such as HRV time, frequency and non-linear domain, EDR etc.

In [89]:
# Main class that extracts features from a dictionary of sorted dataframes and stores to csv
class FeatureExtraction():
    # takes in cleaned ECG data
    def __init__(self, preprocessed_ECG:pd.DataFrame, window_samples:int, sampling_frequency:int):
        self.preprocessed_ECG = preprocessed_ECG
        self.window_samples = window_samples
        self.sampling_frequency = sampling_frequency
        self.feature_extracted_ECG = pd.DataFrame()

    
    def wave_analysis(self, segment:pd.DataFrame, plot=False) -> pd.DataFrame:
        ECG_processed, info = nk.ecg_process(segment.to_numpy(dtype='float64'), sampling_rate=self.sampling_frequency, method='neurokit')
        # calculate the mean and SD of the peak intervals
        peaks = ['ECG_P_Peaks', 'ECG_Q_Peaks', 'ECG_S_Peaks', 'ECG_T_Peaks']
        # Minimum and maximum expected HR (beats per min)
        min_HR = 30
        max_HR = 200
        min_interval = 60e6/max_HR
        max_interval = 60e6/min_HR

        df = pd.DataFrame()
        for peak in peaks:
            intervals = np.diff(np.where(np.array(ECG_processed[peak]==1))) * self.sampling_frequency
            # Remove any intervals that are out of range
            intervals = intervals[(intervals >= min_interval) & (intervals <= max_interval)]

            df[f'{peak}_Interval_Mean'] = [np.mean(intervals)]
            df[f'{peak}_Interval_SD'] = [np.std(intervals)]

        # calculate the average lenght of each wave
        waves = ['P', 'R', 'T']
        max_duration = [120000, 120000, 200000]
        for w, wave in enumerate(waves):
            onsets = np.where(np.array(ECG_processed[f'ECG_{wave}_Onsets']==1))[0]
            offsets = np.where(np.array(ECG_processed[f'ECG_{wave}_Offsets']==1))[0]
            # find index of first element in offsets that is >= first element in onsets
            idx_offset = np.where(offsets >= onsets[0])[0][0]
            # find size of smallest array
            duration_size = min(onsets.size, offsets.size)
            # slice offsets array to start at same index as onset
            offsets = offsets[idx_offset:duration_size]
            # set onset to same length
            onsets = onsets[:duration_size]
            # calculate durations taking into account missed onset detection
            durations = []
            # iterate over elements of both arrays
            i = 0
            j = 0
            while i < len(offsets) and j < len(onsets):
                diff = offsets[i] - onsets[j]
                if diff < 0:
                    i += 1
                else:
                    durations.append(diff)
                    i += 1
                    j += 1
            durations = np.array(durations * self.sampling_frequency)
            # Remove any intervals that are out of range
            durations = durations[(durations <= max_duration[w])]

            duration_mean = np.mean(durations)
            duration_SD = np.std(durations)
            df[f'ECG_{wave}_Duration_Mean'] = duration_mean
            df[f'ECG_{wave}_Duration_SD'] = duration_SD

        return df    

    
    def calc_PSD(self, segment:pd.DataFrame) -> pd.DataFrame:
        # Sum the power across 10 Hz bands from 0 to 200 Hz
        # Compute the power spectrum using a Fast Fourier Transform
        PSD = nk.signal_psd(segment.to_list(), sampling_rate=self.sampling_frequency, method="welch", min_frequency=0.5, max_frequency=200)
        # Create an empty list to store the binned power values
        binned_power = []
        # Set the initial frequency and bin range values
        frequency = 0
        bin_range = 10

        # Loop through the frequency ranges of 10Hz
        while bin_range <= 200:
            # Initialize the total power for the current bin
            total_power = 0
            
            # Loop through the rows of the original dataframe
            for index, row in PSD.iterrows():
                # Check if the frequency falls within the current bin range
                if row['Frequency'] >= frequency and row['Frequency'] < bin_range:
                    # Add the power value to the total power
                    total_power += row['Power']
            
            # Calculate the logarithm of the total power for the current bin and append it to the binned_power list
            if total_power > 0:
                binned_power.append(np.log10(total_power))
            else:
                binned_power.append(-np.inf)
            
            # Increment the frequency and bin range values for the next iteration
            frequency += 10
            bin_range += 10

        # Create a new dataframe with the binned power values and the frequency ranges as the index
        binned_PSD = pd.DataFrame({'Power': binned_power})
        binned_PSD['Frequency Band'] = list(range(10, 201, 10))
        # Convert to columns
        ECG_Frequencies = pd.DataFrame(columns=[f"ECG_FQ_{i}" for i in range(10, 210, 10)])
        for i, column in enumerate(ECG_Frequencies.columns):
            ECG_Frequencies[column] = [binned_PSD.iloc[i]['Power']]
        
        return ECG_Frequencies


    # Main method to extracts features from ECG using neurokit
    def extract(self, ECG:bool=False, EDR:bool=False, show_plot:bool=False):
        # collective epoch analysis
        if ECG:
            print("Extracting Collective Features...")
            # warnings.filterwarnings('ignore') # temporarily supress warnings
            # # automated pipeline for preprocessing an ECG signal
            # ECG_processed, info = nk.ecg_process(self.preprocessed_ECG['ECG'].to_numpy(dtype='float64'), sampling_rate=self.sampling_frequency, method='neurokit')
            # events = np.arange(self.window_samples, self.preprocessed_ECG.shape[0], self.window_samples)
            # epochs = nk.epochs_create(ECG_processed, events=events, sampling_rate=self.sampling_frequency)
            # # calculate ECG Features such as pqrstu intevals etc.
            # ECG_events = nk.ecg_analyze(epochs, sampling_rate=self.sampling_frequency, method='event-related')
            # warnings.filterwarnings('default')

        # individual epoch analysis
        sample_index = 0
        epoch_index = 0
        while sample_index < (len(self.preprocessed_ECG['ECG']) - self.window_samples):
            Utilities.progress_bar('Extracting Individual Features', sample_index, len(self.preprocessed_ECG['ECG']))

            # get segment ECG and stress level from dataframe 
            segment = self.preprocessed_ECG.iloc[sample_index:sample_index + self.window_samples]['ECG']
            stress_level = self.preprocessed_ECG.iloc[sample_index]['Stress Level']
            features = pd.DataFrame({'Stress Level': [stress_level]})
            # extract R-R peaks
            r_peaks_df = nk.ecg_peaks(segment, sampling_rate=self.sampling_frequency, correct_artifacts=True)[0]
            
            sample_index += self.window_samples

            if ECG:
                warnings.filterwarnings('ignore') # temporarily supress warnings
                ECG_processed_segment, info = nk.ecg_process(segment.to_numpy(dtype='float64'), sampling_rate=self.sampling_frequency, method='neurokit')
                if show_plot:
                    nk.ecg_plot(ECG_processed_segment)

                # Calculate ECG_HRV - different heart rate variability metrices.
                HRV_intervals = nk.ecg_intervalrelated(ECG_processed_segment, sampling_rate=self.sampling_frequency)

                # calculate waveform intervals (PQRSTU)
                ECG_intervals = self.wave_analysis(segment)

                # get the binned power spectrum frequencies from the ECG segment
                ECG_frequencies = self.calc_PSD(segment)
                
                # add ECG_event to dataframe (ECG Features such as pqrstu intevals etc.))
                ECG_event = ECG_events.iloc[epoch_index].to_frame().transpose().reset_index()
                ECG_event = ECG_event.drop(['index', 'Label', 'Event_Onset'], axis=1)
                epoch_index += 1

                # concat to dataframe
                features = pd.concat([features, HRV_intervals, ECG_intervals, ECG_Frequencies, ECG_event], axis=1)

            # If not all ECG features desired, obtain HRV and Shannon Entropy
            else:
                np.seterr(divide="ignore", invalid="ignore")
                # skip segment if insufficient peaks are detected (otherwise will cause NK error)
                if int(r_peaks_df[r_peaks_df == 1].sum().iloc[0]) < 4:
                    continue

                # Extract HRV features from R-R peaks, see https://neuropsychology.github.io/NeuroKit/functions/hrv.html 
                # compute HRV - time, frequency and nonlinear indices.
                warnings.filterwarnings('ignore') # temporarily supress warnings
                HRV_time = nk.hrv_time(r_peaks_df, sampling_rate=self.sampling_frequency, show=show_plot)
                HRV_frequency = nk.hrv_frequency(r_peaks_df, sampling_rate=self.sampling_frequency, show=show_plot)
                warnings.filterwarnings('default')

                # compute Shannon Entropy (SE) using signal symbolization and discretization
                # see https://neuropsychology.github.io/NeuroKit/functions/complexity.html#entropy-shannon 
                SE = nk.entropy_shannon(segment, symbolize='A')[0]
                HRV_ShanEn = pd.DataFrame([SE], columns=['HRV_ShanEn'])
                # concat to feature dataframe
                features = pd.concat([features, HRV_time, HRV_frequency, HRV_ShanEn], axis=1)
                
            if EDR:
                # Get ECG Derived Respiration (EDR) and add to the data
                warnings.filterwarnings('ignore') # temporarily supress warnings
                ecg_rate = nk.signal_rate(r_peaks_df, sampling_rate=self.sampling_frequency, desired_length=len(r_peaks_df))
                warnings.filterwarnings('default')
                EDR_sample = nk.ecg_rsp(ecg_rate, sampling_rate=self.sampling_frequency)
                if show_plot:
                    nk.signalplot(segment)
                    nk.signal_plot(EDR_sample)
                EDR_Distance = pd.DataFrame([np.average(nk.signal_findpeaks(EDR_sample)["Distance"])], columns=['EDR_Distance'])
                # concat to dataframe
                features = pd.concat([features, EDR_Distance], axis=1)

            # concat features to main dataframe
            self.feature_extracted_ECG = pd.concat([self.feature_extracted_ECG, features], axis=0, ignore_index=True)
        Utilities.progress_bar('Extracting Neurokit Features', sample_index, sample_index)

In [ ]:
fe = FeatureExtraction(pp.preprocessed_ECG, pp.window_samples, pp.sampling_frequency)
fe.extract(ECG=True, EDR=True, show_plot=False)
Utilities.save_dataframe(fe.feature_extracted_ECG, dataframe_path, 'Feature Extracted HRV_ECG_EDA_20s')

Extracting Collective Features...


NameError: name 'ECG_events' is not defined

## 6. Feature Selection:
* Select desired features, sanity check the values, and save them to Features directory
* Visualise most feature and cross-feature distribution 

In [ ]:
# Each feature will be an object of FEParameter
class FEParameter:
    def __init__(self, name:str, min:float=0.0, max:float=9999):
        self.name = name
        self.min = min
        self.max = max

In [ ]:
# Define FeatureSelection class that is used to visualise and select data
class FeatureSelection():
    def __init__(self, feature_extracted_ECG):
        self.feature_extracted_ECG = feature_extracted_ECG


    # desired_features is a list of FEParameter objects  
    def select(self, desired_features:List[FEParameter]):
        self.selected_features_ECG = self.feature_extracted_ECG[['Stress Level']]
        
        for feature in desired_features:
            out_of_range_count = 0
            # Sanity check: check if feature exists
            if feature.name in self.feature_extracted_ECG.columns:
                # Set value to NaN if it falls outside min and max values.
                for i, value in enumerate(self.feature_extracted_ECG[feature.name]):
                    if (value < feature.min) or (value > feature.max):
                        out_of_range_count += 1
                        self.feature_extracted_ECG.loc[i, feature.name] = np.nan
                # Add column to new selected features
                pd.options.mode.chained_assignment = None
                self.selected_features_ECG[feature.name] = self.feature_extracted_ECG[[feature.name]].copy()
                pd.options.mode.chained_assignment = 'warn'
            else:
                print(f'Error: No such feature "{feature}" in extracted features')
            if out_of_range_count != 0:
                print(f'Feature: {feature.name} is out of range {out_of_range_count}/{len(self.feature_extracted_ECG[feature.name])} segments')

        
    # impute missing values in dataset with mean values of column
    def impute(self):
        pd.options.mode.chained_assignment = None
        self.selected_features_ECG.replace([np.inf, -np.inf], np.nan, inplace=True)
        pd.options.mode.chained_assignment = 'warn'
        imp = SimpleImputer(strategy='mean')
        imp.fit(self.selected_features_ECG)
        self.selected_features_ECG = pd.DataFrame(imp.transform(self.selected_features_ECG), columns=self.selected_features_ECG.columns)


    def visualise(self, plot_type='pairplot'):
        print("Generating plot...")
        if plot_type == 'pairplot':         
            sns.pairplot(data = self.selected_features_ECG, hue='Stress Level', palette=['green', 'yellow', 'red'])
        elif plot_type == 'kdeplot':
            # Create a figure with subplots for each feature
            subplot_size = math.ceil(math.sqrt(len(self.selected_features_ECG.columns)))
            fig = plt.figure(figsize=(20, 8*subplot_size))

            # Loop through each feature and add it to a subplot
            for i, feature in enumerate(self.selected_features_ECG):
                fig.add_subplot(subplot_size, subplot_size, i+1)
                sns.kdeplot(x=feature, data=self.selected_features_ECG, hue='Stress Level', common_norm=False, warn_singular=False, palette=['green', 'yellow', 'red'])
            plt.show()
        else:
            print("Plot type not recognised. Please choose between pairplot, kdeplot")

In [ ]:
# Desired Features to be selected from Feature Extracted ECG.
# See Neurokit2 HRV for more information - https://neuropsychology.github.io/NeuroKit/functions/hrv.html

# Minimum and maximum expected HR (beats per min)
min_HR = 30
max_HR = 200

# MinNN: The minimum of the RR intervals (Parent, 2019; Subramaniam, 2022).
HRV_MinNN = FEParameter('HRV_MinNN', min=60000.0/max_HR, max=60000.0/min_HR)
# MaxNN: The maximum of the RR intervals (Parent, 2019; Subramaniam, 2022).
HRV_MaxNN = FEParameter('HRV_MaxNN', min=60000.0/max_HR, max=60000.0/min_HR)
# MeanNN: The mean of the RR intervals.
HRV_MeanNN = FEParameter('HRV_MeanNN', min=60000.0/max_HR, max=60000.0/min_HR)



# pNN20: The proportion of RR intervals greater than 20ms, out of the total number of RR intervals.
HRV_pNN20 = FEParameter('HRV_pNN20')
# pNN50: The proportion of RR intervals greater than 50ms, out of the total number of RR intervals.
HRV_pNN50 = FEParameter('HRV_pNN50')
# A geometrical parameter of the HRV, or more specifically, the baseline width of the RR intervals distribution 
# TINN: obtained by triangular interpolation, where the error of least squares determines the triangle. 
# It is an approximation of the RR interval distribution.
HRV_TINN = FEParameter('HRV_TINN')
# HTI: The HRV triangular index, measuring the total number of RR intervals divided by the height of the RR intervals histogram.
HRV_HTI = FEParameter('HRV_HTI')

# VLF: The spectral power (W/Hz) of very low frequencies (.0033 to .04 Hz).
HRV_VLF = FEParameter('HRV_VLF', min=0.0, max=9) # hidden due to use of 0.5 Hz high-pass butterworth filter
# LF: The spectral power (W/Hz) of low frequencies (.04 to .15 Hz).
HRV_LF = FEParameter('HRV_LF', max=1.00)
# HF: The spectral power (W/Hz) of high frequencies (.15 to .4 Hz).
HRV_HF = FEParameter('HRV_HF', max=1.00)
# LFHF: The ratio obtained by dividing the low frequency power by the high frequency power.
HRV_LFHF = FEParameter('HRV_LFHF', max=1.00)

# SDNN: The standard deviation of the RR intervals.
# See https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5624990/ for chosen max value.
HRV_SDNN = FEParameter('HRV_SDNN', max=200)
# RMSSD: The square root of the mean of the squared successive differences between adjacent RR intervals. 
# # It is equivalent (although on another scale) to SD1, and therefore it is redundant to report correlations with both (Ciccone, 2017).
# See https://help.welltory.com/en/articles/4413231-what-normal-ranges-and-measurement-standards-we-use-to-interpret-your-heart-rate-variability for chosen max value.
HRV_RMSSD = FEParameter('HRV_RMSSD', max=107)
# The root mean square of successive differences (RMSSD) divided by the mean of the RR intervals (MeanNN).
cv_foldsSD = FEParameter('HRV_cv_foldsSD')
# Shannon Entropy
HRV_ShanEn = FEParameter('HRV_ShanEn')
# Sample Entropy
HRV_SampEn = FEParameter('HRV_SampEn')
# DFA_alpha1: The monofractal detrended fluctuation analysis of the HR signal, corresponding to short-term correlations.
HRV_DFA_alpha1 = FEParameter('HRV_DFA_alpha1')

# EDR Distance: Breathing interval measured in ms.
# Breathing rate range is approximately 12 to 25 per minute - see https://my.clevelandclinic.org/health/articles/10881-vital-signs
EDR_Distance = FEParameter('EDR_Distance', min=2000, max=4167)

In [ ]:
fs = FeatureSelection(fe.feature_extracted_ECG)
#fs = FeatureSelection(Utilities.load_dataframe(f'{dataframe_path}/Feature Extracted HRV_ECG_EDA.csv'))
# desired features for classification
selected_features = [HRV_MinNN, HRV_MaxNN, HRV_MeanNN, HRV_SDNN, HRV_RMSSD, HRV_ShanEn, HRV_SampEn, HRV_DFA_alpha1, HRV_pNN20, HRV_pNN50, EDR_Distance]
fs.select(selected_features)
fs.impute()
fs.visualise(plot_type='kdeplot')
fs.visualise(plot_type='pairplot')
Utilities.save_dataframe(fs.selected_features_ECG, dataframe_path, 'Feature Selected')

KeyboardInterrupt: 

## 7. Traditional Machine Learning Methods
* Prepares data by randomly splitting data into train, test and validation data
* Option for Linear Discriminant Analysis (LDA) for dimension reduction
* Implements the following classification models, with methods for tuning: LDA, Random Forests

In [ ]:
# Structure of dataset that will be passed to Models as object
class Dataset():
    def __init__(self, X, y, X_train, y_train, X_test, y_test, X_valid, y_valid):
        self.X = X
        self.y = y
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.X_valid = X_valid
        self.y_valid = y_valid

In [ ]:
# Utilities for preparing and evaluating classification
class ML_Utilities():  
    # corrects imbalanced dataset using SMOTE, randomizes, splits into test, training and validation data.
    def prepare(selected_features_ECG:pd.DataFrame, train_split:float, test_split:float, validation_split:float=None, normalise=True, LDA_dimension_reduction:bool=False) -> Dataset:     
        label = selected_features_ECG['Stress Level']
        features = selected_features_ECG.loc[:, selected_features_ECG.columns != 'Stress Level']

        print("Before imbalanced label correction (SMOTE):")
        low = label.value_counts()[1.0]/len(label)*100
        print(f'    Low Stress in dataset: {low:.2f}%')
        medium = label.value_counts()[2.0]/len(label)*100
        print(f'    Medium Stress in dataset: {medium:.2f}%')
        high = label.value_counts()[3.0]/len(label)*100
        print(f'    High Stress in dataset: {high:.2f}%')
        
        features, label = SMOTE().fit_resample(features, label)

        # split into test, training and validation data (hold-out validation)
        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=test_split, random_state=15)


        if validation_split != None:
            # Split the training data into training and validation sets
            X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_split/(train_split), random_state=15)
            if normalise:
                X_valid = pd.DataFrame(normalize(X_valid.values, axis=0), columns=X_valid.columns)

        else:
            X_valid = None
            y_valid = None        

        # L2 normalization between 0 and 1 for each feature (if required)
        if normalise:
            features = pd.DataFrame(normalize(features.values, axis=0), columns=features.columns)
            X_train = pd.DataFrame(normalize(X_train.values, axis=0), columns=X_train.columns)
            X_test = pd.DataFrame(normalize(X_test.values, axis=0), columns=X_test.columns)

        # LDA for dimension reduction creates new train and test data
        if LDA_dimension_reduction:
            lda = LinearDiscriminantAnalysis()
            X_train = lda.fit_transform(X_train, y_train)
            X_test = lda.transform(X_test)
            X_valid = lda.transform(X_valid)

        # create data structure
        dataset = Dataset(features, label, X_train, y_train, X_test, y_test, X_valid, y_valid)
        return dataset


    def plot_confustion_matrix(y_test, y_pred):
        # Confusion Matrix
        ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=['Low', 'Medium', 'High'], normalize='true')

In [ ]:
# Custom callback for plotting live loss curve whilst training
class PlotLearning(Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        display.clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label='Train ' + metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='Test ' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [ ]:
# Define Traditional Machine Learning class, which implements different linear and non linear classification methods
class Traditional_ML():
    def __init__(self, dataset:Dataset, database, number_of_cores:int=1):
        self.dataset = dataset
        self.database = database
        self.n_jobs = number_of_cores
        self.model = 'RF'

    # main tune method
    def tune(self):
        if self.model == 'LDA':
            tuning_func = lambda: self.LDA_tuner()
        elif self.model == 'RF':
            tuning_func = lambda: self.RF_tuner()
        elif self.model == 'BT':
            tuning_func = lambda: self.BT_tuner()
        elif self.model == 'NB':
            tuning_func = lambda: self.NB_tuner()
        elif self.model == 'SVM':
            tuning_func = lambda: self.SVM_tuner()
        else:
            raise ValueError(f'Unsupported model: {self.model}')
        tuning_func()


    # main classification method
    def classify(self, cv_folds:int=10):
        if self.model == 'LDA':
            tuning_func = lambda cv_folds: self.LDA_classifier(cv_folds)
        elif self.model == 'BT':
            tuning_func = lambda cv_folds: self.BT_classifier(cv_folds)
        elif self.model == 'RF':
            tuning_func = lambda cv_folds: self.RF_classifier(cv_folds)
        elif self.model == 'NB':
            tuning_func = lambda cv_folds: self.NB_classifier(cv_folds)
        elif self.model == 'SVM':
            tuning_func = lambda cv_folds: self.SVM_classifier(cv_folds)
        else:
            raise ValueError(f'Unsupported model: {self.model}')
        tuning_func(cv_folds)


    def evaluate(self, model, cv_folds:int=0):
        # Cross-validation prediction 
        if cv_folds >= 2:
            X = self.dataset.X
            y = self.dataset.y
            y_pred = cross_val_predict(model, X, y, cv=cv_folds, n_jobs=1)
            # Obtain Accuracy score of cross-validation scores
            acc_cv_folds = accuracy_score(y, y_pred) * 100
            print(f"    {cv_folds} fold cross validation accuracy: {acc_cv_folds.mean():.2f}%")
        
        # Validate on test data
        else:
            X = self.dataset.X_test
            y = self.dataset.y_test
            model.fit(self.dataset.X_train, self.dataset.y_train)
            y_pred_train = model.predict(X=self.dataset.X_train)
            y_pred_test = model.predict(X=X)
            # Obtain accuracy score for train and test data
            acc = accuracy_score(self.dataset.y_train, y_pred_train) * 100
            print(f"    Accuracy on Train Data: {acc:.2f}%")

            acc = accuracy_score(y, y_pred_test) * 100
            print(f"    Accuracy on Test Data: {acc:.2f}%\n")
            y_pred = y_pred_test

         # calculate accuracy for each label
        thr_acc = accuracy_score(y[y == 3.0], y_pred[y == 3.0]) * 100
        tmr_acc = accuracy_score(y[y == 2.0], y_pred[y == 2.0]) * 100
        tlr_acc = accuracy_score(y[y == 1.0], y_pred[y == 1.0]) * 100
        # print results
        print(f"    Accuracy for True High Rate (THR): {thr_acc:.2f}%")
        print(f"    Accuracy for True Medium Rate (TMR): {tmr_acc:.2f}%")
        print(f"    Accuracy for True Low Rate (TLR): {tlr_acc:.2f}%")

        # Plot the confusion matrix
        ML_Utilities.plot_confustion_matrix(y, y_pred)


    def LDA_tuner(self):
        solvers = ['lsqr', 'eigen']
        shrinkages = [None, 'auto', 0.1, 0.5, 0.9]

        for solver, shrinkage in product(solvers, shrinkages):
            clf = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage).fit(self.dataset.X_train, self.dataset.y_train)
            clf_score = clf.score(self.dataset.X_test, self.dataset.y_test)
            print(f'Score for solver={solver}, shrinkage={shrinkage}: {clf_score}')


    def LDA_classifier(self, cv_folds):
        clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=0.5)

        cv_folds_scores = cross_val_score(clf, self.dataset.X_train, self.dataset.y_train, cv=cv_folds)
        mean_cv_folds_score = cv_folds_scores.mean()

        clf.fit(self.dataset.X_train, self.dataset.y_train)
        test_score = clf.score(self.dataset.X_test, self.dataset.y_test)

        print(f'Mean cross-validation score for LDA: {mean_cv_folds_score}')
        print(f'Test score for LDA: {test_score}')
        

    def RF_tuner(self):
        print("\nRandom Forest Tuner:")
        max_features = [1, 'sqrt', 'log2']
        max_depths = [None, 2, 3, 4, 5]
        for f, d in product(max_features, max_depths): # with product we can iterate through all possible combinations
            rf = RandomForestClassifier(n_estimators=100, 
                                        criterion='entropy', 
                                        max_features=f, 
                                        max_depth=d, 
                                        n_jobs=self.n_jobs,
                                        random_state=15)
            rf.fit(self.dataset.X_train, self.dataset.y_train)
            self.dataset.y_pred = rf.predict(X=self.dataset.X_test)
            print(' Classification accuracy on test set with max features = {} and max_depth = {}: {:.3f}'.format(f, d, accuracy_score(self.dataset.y_test,self.dataset.y_pred)))

        # chosen max_depth of None, max_features of log2
        n_estimators = [50, 100, 150, 200, 500]
        criterion = ['gini', 'entropy', 'log_loss']
        for n, c  in product(n_estimators, criterion):
            rf = RandomForestClassifier(n_estimators=n,
                                        criterion=c, 
                                        max_features=1, 
                                        max_depth=None, 
                                        n_jobs=self.n_jobs,
                                        random_state=15)
            rf.fit(self.dataset.X_train, self.dataset.y_train)
            self.dataset.y_pred = rf.predict(X=self.dataset.X_test)
            print(' Classification accuracy on test set with n_estimators = {} and criterion = {}: {:.3f}'.format(n, c, accuracy_score(self.dataset.y_test,self.dataset.y_pred)))
            ML_Utilities.plot_confustion_matrix(self.dataset.y_test, self.dataset.y_pred)


    # Post-tuned Random Forest Classifier
    def RF_classifier(self, cv_folds:int=10):
        # source: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
        print("\nRandom Forest Classifier:")
        
        if (self.database == 'Spider'):
            rf = RandomForestClassifier(n_estimators=100,
                                        criterion='log_loss', 
                                        max_features='log2', 
                                        max_depth=None, 
                                        n_jobs=self.n_jobs,
                                        random_state=15)
        elif (self.database == 'BrainPatch'):
            rf = RandomForestClassifier(n_estimators=500,
                                        criterion='gini',
                                        max_features=1,
                                        max_depth=None,
                                        n_jobs=self.n_jobs,
                                        random_state=15)

        self.evaluate(rf, cv_folds)


    # Naive Bayes Classifier
    def NB_tuner(self):
        print("\nNaive Bayes Classifier:")
        # create a Multinomial Naive Bayes classifier
        nb = MultinomialNB()

        # set up a parameter grid to search
        param_grid = {'alpha': [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]}

        # perform a grid search to find the best hyperparameters
        grid_search = GridSearchCV(nb, param_grid=param_grid, cv=5, iid=False)
        grid_search.fit(X_train, y_train)

        # print the best hyperparameters
        print('Best hyperparameters: {}'.format(grid_search.best_params_))

        # make predictions on the test data using the best model
        y_pred = grid_search.predict(X_test)

        # calculate and print the accuracy of the best model
        print('Classification accuracy on test set with best hyperparameters: {:.3f}'.format(accuracy_score(y_test, y_pred)))

    # Post tuned Naive Bayes Classifer
    def NB_classifier(self, cv_folds:int=10):
        # create a Gaussian Naive Bayes classifier
        nb = GaussianNB()

        # fit the classifier to the training data
        nb.fit(self.dataset.X_train, self.dataset.y_train)

        # make predictions on the test data
        self.dataset.y_pred = nb.predict(self.dataset.X_test)

        # calculate and print the accuracy of the classifier
        print('Classification accuracy on test set with Naive Bayes: {:.3f}'.format(accuracy_score(self.dataset.y_test, self.dataset.y_pred)))


    # Support Vector Machine Classifier 
    def SVM_tuner(self):
        print("\nSupport Vector Machine Classifier:")
        # Define a list of hyperparameters to tune
        kernels = ['linear', 'poly', 'rbf', 'sigmoid, 'precomputed']
        Cs = [0.1, 1, 10]

        # Iterate through all combinations of hyperparameters
        for kernel, C in product(kernels, Cs):
            svm = SVC(kernel=kernel, C=C, random_state=15)
            svm.fit(self.dataset.X_train, self.dataset.y_train)
            y_pred = svm.predict(self.dataset.X_test)
            accuracy = accuracy_score(self.dataset.y_test, y_pred)
            print(' SVM with kernel = {} and C = {}: accuracy = {:.3f}'.format(kernel, C, accuracy))
            ML_Utilities.plot_confustion_matrix(self.dataset.y_test, y_pred)

    # Post-tuned Support Vector Machine Classifier 
    def SVM_classifier(self, cv_folds:int=10):
        svm = SVC(kernel='linear', C=1, random_state=15)
        # Perform cross-fold validation
        scores = cross_val_score(svm, self.dataset.X_train, self.dataset.y_train, cv=5)
        # Print the mean and standard deviation of the cross-validation scores
        print("Cross-validation accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
        # Make predictions using cross-fold validation
        y_pred = cross_val_predict(svm, self.dataset.X_test, self.dataset.y_test, cv=5)
        print('Classification accuracy with kernel = {} and C = {}: {:.3f}'.format(svm.kernel, svm.C, accuracy_score(self.dataset.y_test, y_pred)))
        ML_Utilities.plot_confustion_matrix(self.dataset.y_test, y_pred)


    # Tuning Bagging Classifier
    def BT_tuner(self):
        print("\nBagging Decision Trees Classifier:")
        # Define the hyperparameters to tune
        param_grid = {
            'n_estimators': [10, 50, 100],
            'max_samples': [0.5, 1.0],
            'max_features': [0.5, 1.0],
            'bootstrap': [True, False],
            'bootstrap_features': [True, False]
        }

        bt_clf = BaggingClassifier(estimator=SVC())

        # Perform grid search to find the best hyperparameters
        grid_search = GridSearchCV(bt_clf, param_grid=param_grid, cv=2, n_jobs=1, verbose=2)
        grid_search.fit(self.dataset.X_train, self.dataset.y_train)

        # Print the best hyperparameters
        print(" Best hyperparameters: ", grid_search.best_params_)


    # Post-tuned Bagged Trees Classifier
    def BT_classifier(self, cv_folds:int=10):
        bt = BaggingClassifier()
        self.evaluate(bt, cv_folds)

In [ ]:
# selected_features_ECG = Utilities.load_dataframe(f'{dataframe_path}Feature Selected.csv')

dataset = ML_Utilities.prepare(fs.selected_features_ECG, train_split=0.70, test_split=0.30, normalise=False)

tml = Traditional_ML(dataset, DATABASE, number_of_cores=1)

# 'LDA' for Linear Discriminant Analysis
# 'RF' for Random Forest
# 'NB' for Naive Bayes
# 'BT' for Bagged Trees
# 'SVM' for Support Vector Machine
tml.model = 'LDA'
tml.tune()
tml.classify(cv_folds=5)

## 8. Convolutional Neural Network
* Prepares data by converting the sorted data into a 3 dimensional shape
* 1D Convolutional Neural Network tuning and regression model
* Regression is converted to discrete classification

In [ ]:
# 1 dimensional Convolutional Deep Neural Network
class CNN1D():
    def __init__(self, dataset:Dataset, database):
        self.dataset = dataset
        self.database = database

    # prepare data by setting shape to 3 dimensions
    def prepare(self):
        # get shape of data
        self.n_features, self.n_classes = self.dataset.X_train.shape[1], self.dataset.y_train.nunique()

        # Format to a NumPy array of 32-bit floating-point numbers and reshape to a 3D array with dimensions (number of samples, number of features, 1)
        self.dataset.X = np.asarray(self.dataset.X).astype(np.float32).reshape(-1, self.n_features, 1)
        self.dataset.X_test = np.asarray(self.dataset.X_test).astype(np.float32).reshape(-1, self.n_features, 1)
        self.dataset.X_train = np.asarray(self.dataset.X_train).astype(np.float32).reshape(-1, self.n_features, 1)
        self.dataset.y = np.asarray(self.dataset.y).astype(np.float32)
        self.dataset.y_test = np.asarray(self.dataset.y_test).astype(np.float32)
        self.dataset.y_train = np.asarray(self.dataset.y_train).astype(np.float32)

        # convert to one-hot encoding
        self.dataset.y = to_categorical((self.dataset.y-1), self.n_classes)
        self.dataset.y_test = to_categorical((self.dataset.y_test-1), self.n_classes)
        self.dataset.y_train = to_categorical((self.dataset.y_train-1), self.n_classes)
    
    
    def convert_to_labels(self, y_true, y_pred):
        # convert from regression to class label
        max_indices = np.argmax(y_pred, axis=1)
        result = np.eye(y_pred.shape[1])[max_indices]
        # Convert the one-hot vectors back to their original labels
        labels = [1.0, 2.0, 3.0]
        label_indices = np.argmax(result, axis=1)
        y_pred_l = [labels[i] for i in label_indices]

        label_indices = np.argmax(y_true, axis=1)
        y_true_l = [labels[i] for i in label_indices]     

        return y_true_l, y_pred_l 
    

    def tune(self):
        # hyperparameters
        learning_rate, epochs, batch_size = 0.00001, 20, 32

        # Define the model architecture
        self.model = Sequential([
            # Input layer
            InputLayer(input_shape=(self.n_features, 1)),
            
            # 1D Convolutional layer
            Conv1D(filters=32, kernel_size=3, activation='relu'),
            
            # Max Pooling layer
            MaxPooling1D(pool_size=2),
            
            # Flatten layer
            Flatten(),
            
            # Dropout layer
            Dropout(0.5),
            
            # Fully Connected layer 1
            Dense(units=128, activation='relu'),
            
            # Fully Connected layer 2 to number of class outputs
            Dense(units=self.n_classes, activation='softmax')
        ])

        # print model architecture summary
        self.model.summary()

        # Compile the model using the Adam Optimiser
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        self.model.fit(self.dataset.X_train, self.dataset.y_train, epochs=epochs, batch_size=batch_size, validation_data=(self.dataset.X_test, self.dataset.y_test), callbacks=[PlotLearning()])

        # Evaluate the model
        score = self.model.evaluate(self.dataset.X_test, self.dataset.y_test, batch_size=32)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        y_pred = self.model.predict(self.dataset.X_test)
        y_true_l, y_pred_l = self.convert_to_labels(self.dataset.y_test, y_pred)
        # plot confusion matrix
        ML_Utilities.plot_confustion_matrix(y_true_l, y_pred_l)


    # post-tuned classifier with k-fold evaluation and confusion matrix
    def predict(self, cv_folds=5):
        print(f"Conducting {cv_folds} Cross Validation Folds")

        # split the dataset into training and validation sets using KFold
        kf = KFold(n_splits=cv_folds, shuffle=True, random_state=15)
        scores = []
        X_val, y_val = 0, 0
        for fold, (train_index, val_index) in enumerate(kf.split(self.dataset.X)):
            print('Fold', fold+1)
            X_train, y_train = self.dataset.X[train_index], self.dataset.y[train_index]
            X_val, y_val = self.dataset.X[val_index], self.dataset.y[val_index]

            # train your model on the current fold
            self.model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[PlotLearning()])

            # Evaluate the model on the test set for this fold and store the accuracy score
            score = self.model.evaluate(X_val, y_val, verbose=0)
            scores.append(score)

        # Calculate the average accuracy score across all folds
        average_test_loss = sum(scores[0]) / len(scores[0])
        average_accuracy = sum(scores[1]) / len(scores[1])

        print(f'Average test Loss: {average_test_loss}')
        print(f'Average test accuracy: {average_accuracy}')

In [ ]:
# pp.segment(window_length_s=30, overlap=0.1) # segment data with a sliding window of length to be 0.8s which is average heart beat
# pp.clean() 
# pp.create_2d() # converts data to 2d vector (required for CNN)
# Utilities.save_dataframe(pp.preprocessed_ECG_2d, dataframe_path, 'Preprocessed-2d-30.s')

dataset = ML_Utilities.prepare(pp.preprocessed_ECG_2d, train_split=0.70, test_split=0.30, normalise=True)

# Tune and classify the one dimensional convolutional deep neural network
cnn = CNN1D(dataset, DATABASE)
cnn.prepare()
cnn.tune()
cnn.predict(2)
